Agent Example

In [136]:
from pymongo import MongoClient
import os
from langchain_openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import MongoDBAtlasVectorSearch

In [138]:
# query = "What are articles on Apple"

# results = vector_search.similarity_search_with_score(
#     query=query, k=5
# )

# # Display results
# for result in results:
#     print(result)

Below is for scenario 1. Scenario 2 and 3 are just dummy functions

In [139]:
from langchain.prompts import PromptTemplate

DB_NAME = "news_articles"
COLLECTION_NAME = "newsArticleCollection"
ATLAS_VECTOR_SEARCH_INDEX_NAME = "vector_index"
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

prompt_template = """ 
    Answer user's questions with the information you know.

    {context}

    Question: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

# chat completion llm
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name='gpt-3.5-turbo-0125',
    temperature=0.0
)

vector_search = MongoDBAtlasVectorSearch.from_connection_string(
    os.getenv("MONGODB_URI"),
    DB_NAME + "." + COLLECTION_NAME,
    OpenAIEmbeddings(model="text-embedding-3-large", dimensions=1536),
    index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME,
    text_key = "content"
)

qa_retriever = vector_search.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5},
)

# retrieval qa chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever= qa_retriever,
    # return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT},
    verbose=True
)

Scenario 2 and 3 dummy functions

In [140]:

# Initialize MongoDB
mongo_client = MongoClient(os.getenv("MONGODB_URI"))
db = mongo_client.get_database("news_articles")
collection = db.get_collection("newsArticleCollection")

query_embeddings = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=1536) # model used to embed user queries

def vector_search(query, collection):
    query_embedding = query_embeddings.embed_query(query)
    pipeline = [
        {
            "$vectorSearch": { # $vectorSearch is the specific function name
                "index": "vector_index", # The search index I created on MongoDB
                "queryVector": query_embedding, # The embedded query from the user that is used for searching
                "path": "embedding", # The relevant field of the document that is used for searching (in this case the full text of the news article)
                "limit": 5, # How many results you want the vectorSearch to show
                "numCandidates": 100 # How many documents you want vectorSearch to consider when searching
            }
        }
    ]
    
    results = collection.aggregate(pipeline) # executing the search
    return list(results) # compile results into a list

In [141]:
def find_date(query):
    return """do not summarize original result: Here is the article for the date given: https://www.cnbc.com/2024/03/19/european-markets-live-updates-stocks-news-data-and-fed-meeting.html 
European markets were flat Tuesday as global investors look ahead to the start of the U.S. Federal Reserve’s two-day policy meeting.

The pan-European Stoxx 600
 was down 0.01% in early afternoon deals, with sectors trading in mixed territory. Autos stocks rose 1%, while utilities were down 0.6%.

U.S. stocks opened mixed ahead of the Fed’s meeting in Washington. Recent inflation reports could prompt the Fed to signal that interest rates will remain higher for longer than expected. Fed funds futures currently forecast a 99% likelihood that the Fed will leave benchmark interest rates unchanged this week, according to the CME FedWatch Tool."""

def recommend(query):
    get_knowledge = vector_search(query, collection) # Get the output from MongoDB after vector searching

    search_result = ''
    for result in get_knowledge:
        search_result += f"Title: {result.get('title', 'N/A')}, Link: {result.get('url', 'N/A')}, \n"
    return search_result

Different Scenarios for QA handling 

In [142]:
from langchain.agents import Tool

tools = [
    #scenario #2
    #This works e.g. "Give me apple news on 13 March 2024" 
    Tool(
        name='Date period',
        func=find_date,
        description=(
            'use this tool when user mentioned the date. it could be 30 May 2000 or 24/12/19, do not summarize observation. just return full observation result'
        )
    ),
    #scenario #1
    #This works e.g. "What is Apple doing"
    Tool(
        name='Knowledge Base (Ask)',
        func=qa.run,
        description=(
            'use this tool when answering questions related to the news asked by users. if user mentioned date or recommendation, do not use this tool, do not summarize observation. just return full observation result'
        )
    ),
    
    #scenario #3
    #This works e.g. "Recommend me Apple news"
    Tool(
        name='Knowledge Base (Recommend)',
        func=recommend,
        description=(
            'use this tool when answering news recommendations. such as when user say "recommend" or "suggest" or similar words, just return full observation result'
        )
    )
]

In [143]:
from langchain.agents import initialize_agent
# chat completion llm
llm2 = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name='gpt-3.5-turbo-1106',
    temperature=0.0
)

# conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)

agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm2,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory
)

In [74]:
agent("What is Saudi Arabia doing")



> Entering new AgentExecutor chain...
```json
{
    "action": "Knowledge Base (Ask)",
    "action_input": "What is Saudi Arabia doing"
}
```

> Entering new RetrievalQA chain...

> Finished chain.

Observation: Saudi Arabia is set to see a $100 billion investment over the next six years, with an aim to become a leader in the field of electronics and advanced industrials. This investment will go towards enhancing the capabilities of the technological sector in Saudi Arabia.
Thought:```json
{
    "action": "Final Answer",
    "action_input": "Saudi Arabia is set to see a $100 billion investment over the next six years, with an aim to become a leader in the field of electronics and advanced industrials. This investment will go towards enhancing the capabilities of the technological sector in Saudi Arabia."
}
```

> Finished chain.


{'input': 'What is Saudi Arabia doing',
 'chat_history': [],
 'output': 'Saudi Arabia is set to see a $100 billion investment over the next six years, with an aim to become a leader in the field of electronics and advanced industrials. This investment will go towards enhancing the capabilities of the technological sector in Saudi Arabia.'}

In [75]:
agent("Give me news on date 13 March 2024 (https://www.cnbc.com/2024/03/19/european-markets-live-updates-stocks-news-data-and-fed-meeting.html )")



> Entering new AgentExecutor chain...
```json
{
    "action": "Date period",
    "action_input": "13 March 2024"
}
```
Observation: do not summarize original result: Here is the article for the date given: https://www.cnbc.com/2024/03/19/european-markets-live-updates-stocks-news-data-and-fed-meeting.html 
European markets were flat Tuesday as global investors look ahead to the start of the U.S. Federal Reserve’s two-day policy meeting.

The pan-European Stoxx 600
 was down 0.01% in early afternoon deals, with sectors trading in mixed territory. Autos stocks rose 1%, while utilities were down 0.6%.

U.S. stocks opened mixed ahead of the Fed’s meeting in Washington. Recent inflation reports could prompt the Fed to signal that interest rates will remain higher for longer than expected. Fed funds futures currently forecast a 99% likelihood that the Fed will leave benchmark interest rates unchanged this week, according to the CME FedWatch Tool.
Thought:```json
{
    "action": "Final Answe

{'input': 'Give me news on date 13 March 2024 (https://www.cnbc.com/2024/03/19/european-markets-live-updates-stocks-news-data-and-fed-meeting.html )',
 'chat_history': [HumanMessage(content='What is Saudi Arabia doing'),
  AIMessage(content='Saudi Arabia is set to see a $100 billion investment over the next six years, with an aim to become a leader in the field of electronics and advanced industrials. This investment will go towards enhancing the capabilities of the technological sector in Saudi Arabia.')],
 'output': 'European markets were flat on 13 March 2024 as global investors look ahead to the start of the U.S. Federal Reserve’s two-day policy meeting. The pan-European Stoxx 600 was down 0.01% in early afternoon deals, with sectors trading in mixed territory. Autos stocks rose 1%, while utilities were down 0.6%. U.S. stocks opened mixed ahead of the Fed’s meeting in Washington. Recent inflation reports could prompt the Fed to signal that interest rates will remain higher for long

In [144]:
agent("Recommend me some Apple news")



> Entering new AgentExecutor chain...
```json
{
    "action": "Knowledge Base (Recommend)",
    "action_input": "Apple news"
}
```
Observation: Title: This startup taps quantum, AI to fast-track discovery of new materials, Link: https://thenextweb.com/news/this-startup-taps-quantum-ai-to-fast-track-discovery-of-new-materialsTitle: The UK is improving productivity through new technologies and industrial digital transformation, Link: https://www.businessinsider.com/sc/how-the-uk-is-helping-the-manufacturing-sector-improve-productivityTitle: Inside Finland’s state-of-the-art quantum computing hardware ecosystem, Link: https://thenextweb.com/news/finland-quantum-computing-hardware-ecosystemTitle: Saudi Arabia to invest $100bn to create advanced robotics industry, Link: https://readwrite.com/saudi-arabia-to-invest-100bn-to-create-advanced-robotics-industry/Title: New AI and 5G advancements will usher in the era of edge computing on smartphones, autonomous cars, and more, Link: https://www

{'input': 'Recommend me some Apple news',
 'chat_history': [],
 'output': 'This startup taps quantum, AI to fast-track discovery of new materials, The UK is improving productivity through new technologies and industrial digital transformation, Inside Finland’s state-of-the-art quantum computing hardware ecosystem, Saudi Arabia to invest $100bn to create advanced robotics industry, New AI and 5G advancements will usher in the era of edge computing on smartphones, autonomous cars, and more'}

In [22]:
agent('2+2 =?')



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "4"
}
```

> Finished chain.


{'input': '2+2 =?',
 'chat_history': [HumanMessage(content='What is Saudi Arabia doing'),
  AIMessage(content="Saudi Arabia is set to see a $100 billion investment over the next six years, with a focus on becoming a leader in the field of electronics and advanced industrials. The country's Public Investment Fund company Alat was recently launched, and partnerships with companies like Softbank Group, Carrier Corporation, Dahua Technology Ltd, and The Saudi Technology and Security Comprehensive Control Company (Tahakom) have been announced. The investments will go towards enhancing the technological sector in Saudi Arabia and promoting sustainability measures in manufacturing."),
  HumanMessage(content='Give me news on date 13 March 2024 (https://www.cnbc.com/2024/03/19/european-markets-live-updates-stocks-news-data-and-fed-meeting.html )'),
  AIMessage(content='European markets were flat on 13th March 2024 as global investors looked ahead to the start of the U.S. Federal Reserve’s two-d

OpenAI example

In [ ]:

# import os
# from openai import OpenAI

# system_message = {"role": "system", 
#                   "content": """You are a bot that will either provide news recommendations, news summaries or answer questions related to the news asked by users.
#     You are speaking to a professional so keep the answer informative and concise.

#     You are given an article(s) to summarize. Please respond with the following using information given.
#     Title: <Title Name>
#     Website Link: <Link of Website>
#     Date of Article: <Get the latest date of publication>
#     Names to note: <Names of Company and people mentioned within the article>
#     Key Topic: <Key topic of this article>
#     Sentiment: <conduct sentiment analysis and let them know the sentiment>
#     Trends & Statistics: <Include any trends and statistics found, make it short and do not repeat it in summary>
#     Summary: <The summarised version of the article>

#         Assistant:"""}

# conversation = []
# conversation.append(system_message)

# client = OpenAI(
#     # defaults to os.environ.get("OPENAI_API_KEY")
#     api_key=os.getenv("OPENAI_API_KEY"),
# )

# def user_query(query):
#     #add user query to conversation for gpt's response 
#     conversation.append({"role": "user", "content": query})
#     response = client.chat.completions.create(
#         model="gpt-3.5-turbo-1106",
#         messages= conversation
#         # temperature=0.7,
#     )
#     #add bot reply to conversation
#     conversation.append({"role": "assistant", "content": response.choices[0].message.content})
#     return response.choices[0].message.content.strip()

# from newsMongo import urlScrapeAndStore
# #link test
# url = ['<https://www.cnbc.com/2023/04/30/cloud-providers-amazon-microsoft-and-google-face-spending-cuts.html>']
# output = ''
# articles = []
# for x in url:
#     article = urlScrapeAndStore(x)
#     articles.append(article)
#     output = user_query(str(articles))
#     print("Here is output \n")
#     print(output)
#     print("\n")

# # response = user_query("""Summarise this Coral Bay Nickel Corporation donated uniforms and medical kits to Palawan’s barangay health workers on Monday, an initiative enhancing local healthcare support.

# # The Provincial Information Office said Vice Governor Leoncio Ola and Board Member Marivic Roxas, chairperson of the Committee on Health and Social Services of the Sangguniang Panlalawigan, received the donations.

# # Ernesto Llacuna, community relations manager of CBNC, facilitated the distribution of approximately 2,000 barangya health workers (BHW) uniforms, 150 BP monitors, and 150 thermometers to health workers in the southern region of the province.

# # Attendees included Board Members Juan Antonio Alvarez and Luzviminda Bautista, Dr. Justyn Barbosa, Medical Specialist IV, representing Provincial Health Officer Dr. Faye Erika Q. Labrador, and CVHW Manager Jenevil V. Tombaga.
# #                       """)
# # print(response)

# # print("Here is the conversation with template but with user and chatbot history \\n")
# # print(conversation)

# #can delete/edit role: system basically the template by del conversation[0] or conversation[0]['content'] = new template
# # del conversation[0]
# conversation[0]['content'] = """new template"""

# # print("Here is the conversation without template but with user and chatbot history \n")
# # print(conversation)